# Adding memoization via decorators

While memoization makes our code a lot faster, it also clutters it a bit with all the steps for retrieving values from the memo or storing them there.
This can make the code harder to read, and it also distracts from the central flow inside the function.
Python provides a special mechanism to modify functions through other functions, which are called **decorators**.
With decorators, we can add memoization to a function without changing any of its internals.

First, take one more look at the recursive implementation of Levenshtein distance, with memoization.

In [ ]:
def levenshtein_distance(s, t):
    """Calculate Levenshtein distance between s and t."""
    return cost((len(s), len(t)), construct_grid(s, t))

In [ ]:
def construct_grid(s, t):
    """Compute the default cost for each edge."""
    # grid is now a dictionary instead of a list
    grid = {(x,y): {}
            for x in range(len(s) + 1)
            for y in range(len(t) + 1)}
    for x, y in grid:
        # add deletion edge if there is a node to the left
        if x > 0:
            grid[(x,y)][(x-1, y)] = 1
        # add insertion edge is there is a node above
        if y > 0:
            grid[(x,y)][(x, y-1)] = 1
        # add substitution edge; check if cost is 0
        if x > 0 and y > 0:
            grid[(x,y)][(x-1, y-1)] =\
                0 if s[x-1] == t[y-1] else 1
    return grid

In [ ]:
def cost(node, grid):
    """Calculate the cost of the optimal path to node through the grid."""
    if node == (0, 0) or memo.get(node) is not None:
        return memo[node]
    else:
        lowest = min([cost(neighbor, grid) + edge_cost
                      for neighbor, edge_cost in grid[node].items()])
        memo[node] = lowest
        return lowest

All the memoization steps take place inside `cost`, where they are intermingled with the steps for computing the cost as the sum of the cost to get to the neighbor and the cost of the edge from a neighbor.

Instead, we can factor out all the memoization steps into a special function `memoize`.
Intuitively, `memoize` will be used to sandwich `cost` between pieces of code that handle the memoization.
Here's what this looks like.

In [ ]:
def memoize(cost_function):
    """Add memoization to the cost function."""
    memo = {(0, 0): 0}
    def cost_wrapper(node, grid):
        if node == (0, 0) or node in memo:
            return memo[node]
        else:
            lowest = cost_function(node, grid)
            memo[node] = lowest
            return lowest
    return cost_wrapper

@memoize
def cost(node, grid):
    """Calculate the cost of the optimal path to node through the grid.
    
    This version has no memoization."""
    if node == (0, 0):
        return 0
    else:
        lowest = min([cost(neighbor, grid) + edge_cost
                      for neighbor, edge_cost in grid[node].items()])
        return lowest

And it does indeed add memoization, as is witnessed by how little time it takes to compute the example below.

In [ ]:
s = "supercalifragilisticexpialidocious"
t = s
print(f"Levenshtein distance of \"{s}\" and \"{t}\" is {levenshtein_distance(s, t)}")

But how does it work?
The line `@memoize` tells Python that we want to use the function `memoize` to construct a modified version of `cost`.
After that, `memoize` will take the `cost` function as an argument and use it to construct a new function `cost_wrapper`.
First it defines `memo` as `{(0, 0): 0}`.
This is followed up by the function definition for `cost_wrapper`, right inside `memoize`:

- Just like `cost`, it takes the arguments `node` and `grid`.
- If there is already a stored value for `node` in `memo`, `cost_wrapper` returns that.
- Otherwise, `cost_wrapper` uses `cost` to compute the value.
  Once it has been computed, it gets added to `memo` and is returned as the output.
  
After the definition of `cost_wrapper`, `memoize` ends with `return cost_wrapper` to indicate that the function we fed into `memoize` - i.e. `cost` - should be replaced by `cost_wrapper` instead.

Your head is probably swirling at this point, so let's go through this step by step, replacing pieces of code in a manner that is similar to what Python is doing under the hood.

In [ ]:
@memoize
def cost(node, grid):
    ...
# call memoize with cost as the argument, and replace cost by the output of memoize
cost = memoize(cost)

# what is memoize(cost)? it's whatever is returned by memoize: cost_wrapper!
cost = cost_wrapper

# what is cost_wrapper?
def cost_wrapper(node, grid):
    if node == (0, 0) or node in memo:
        return memo[node]
    else:
        lowest = cost(node, grid)
        memo[node] = lowest
        return lowest
    
# we can make the computation of lowest more explicit based on cost
def cost_wrapper(node, grid):
    if node == (0, 0) or node in memo:
        return memo[node]
    else:
        lowest = min([cost(neighbor, grid) + edge_cost
                      for neighbor, edge_cost in grid[node].items()])
        memo[node] = lowest
        return lowest

# but every instance of cost is now cost_wrapper,
# so the definition is actually
def cost_wrapper(node, grid):
    if node == (0, 0) or node in memo:
        return memo[node]
    else:
        lowest = min([cost_wrapper(neighbor, grid) + edge_cost
                      for neighbor, edge_cost in grid[node].items()])
        memo[node] = lowest
        return lowest

This isn't quite how Python handles decorators, but it is a reasonable approximation of the general principle. Now compare this final version of `cost_wrapper` with the memoized version of `cost` that we had in the earlier notebook.

In [ ]:
def cost(node, grid):
    """Calculate the cost of the optimal path to node through the grid."""
    if node == (0, 0) or node in memo:
        return memo[node]
    else:
        lowest = min([cost(neighbor, grid) + edge_cost
                      for neighbor, edge_cost in grid[node].items()])
        memo[node] = lowest
        return lowest

The two are exactly the same!
The only cosmetic difference is that every instance of `cost` is replaced with `cost_wrapper`.
So by using the `memoize` decorator, we have automatically translated the version of `cost` without memoization into the version that does memoize.

But the two are actually not quite identical, the decorator approach has one advantage.
As discussed in the other expansion unit, our memoized implementation of the Levenshtein distance requires `memo` to be instantiated outside the function, or to be a global variable.
With the decorator, we do not need to worry about this.
Since `memo` is instantiated inside `memoize`, it is accessible to any function that is defined within `memoize`.
In other words, `cost_wrapper` (and only `cost_wrapper`) can access `memo` at any time.
The user does not need to worry about creating `memo` first, it is all part of the decoration process.

Decorators have many applications besides memoization.
Any kind of general purpose code that you want to use with a variety of functions is a good candidate for a decorator.
For instance, checking that a function's arguments are of the right type, or supplying more complex default values for arguments.
But we'll stop here because odds are that this has already been plenty to absorb.
Don't worry if it still seems very opaque too you, decorators are a very advanced feature of Python and it takes quite a while to get the hang of them.
Appropriate usage of decorators is a telltale sign that somebody has serious Python programming chops.

## Bullet point summary

- Decorators are used to modify and/or expand other functions.
- Intuitively, a decorator wraps additional code around an existing function.
- More technically, a decorator is a function that takes a function as input and returns another function as output.
- Decorators are invoked with `@` and must appear right above the definition of the function that should be decorated.

```python
@some_decorator
def function_to_be_decorated(arg1, ..., argn):
```

- A decorator can contain arbitrary code, but it must contain the definition of some function `foo`, which is also the decorator's return value.